In [1]:
import pandas as pd
import numpy as np
import re
import psycopg2
from sqlalchemy import create_engine
import config

In [2]:
database=config.database
user=config.user
password=config.password
host=config.db_host
port=config.port

# psycopg2 connection
conn_string = f"host={host} dbname={database} user={user} password={password}"
conn = psycopg2.connect(conn_string)

# Query the pitching data
cur = conn.cursor()
cur.execute('SELECT * FROM "Pitching_data";')
query_results_pitching = cur.fetchall()

# Extract column headers
columns = [desc[0] for desc in cur.description]
#Tranform the data into CSV
pitching_df=pd.DataFrame(query_results_pitching, columns=columns)

# Close connections
cur.close()

# Query the data
cur = conn.cursor()

cur.execute('SELECT * FROM "Batter_data";')
query_results_batting = cur.fetchall()

# Extract column headers
columns = [desc[0] for desc in cur.description]

batting_df=pd.DataFrame(query_results_batting,columns=columns)

# Close connections
cur.close()
conn.close()

In [3]:
# Extract Player name of the Batting column
batting_df['Player Name'] = batting_df['Batting'].str.extract(r'^\s*(\w+\s+\w+)\s+[\w-]+$')
# Extract the position of the Batting column
batting_df['Position'] = batting_df['Batting'].str.extract(r'^\s*\w+\s+\w+\s+([\w-]+)$')

# Drop column 'Batting'
batting_df = batting_df.drop(columns=['Batting'])

# Reorder columns: move the last two columns to the front
cols = batting_df.columns.tolist()  # Get a list of columns
reordered_cols = cols[-2:] + cols[:-2]  # Move last two to the front
batting_df = batting_df[reordered_cols]

# Use loc for assignment with boolean indexing to replace mising venue
batting_df.loc[batting_df["Attendace"].str.startswith("Venue: ", na=False), "Venue"] = (
    batting_df.loc[batting_df["Attendace"].str.startswith("Venue: ", na=False), "Attendace"]
)
batting_df.loc[batting_df["Attendace"].str.startswith("Venue: ", na=False), "Attendace"] = np.nan

In [4]:
#Clean the venue name
batting_df["Venue"]=batting_df["Venue"].str.replace("Venue: ","",regex=True)

#Clean the attendance
batting_df["Attendace"]=batting_df["Attendace"].str.replace("Attendance: ","",regex=True)

#Extract the day of the week from the date
batting_df['Week-Day'] = batting_df['Date'].str.extract(r'^\s*(\w+)\,\s\w+\s\d+\,\s\d+$')

# Convert to datetime
batting_df['Date'] = pd.to_datetime(batting_df['Date'], format='%A, %B %d, %Y')

In [5]:
# Extract only the time
batting_df['Start Time'] =  batting_df['Time'].str.extract(r'(?i)\b(\d{1,2}:\d{2}\s?[ap]\.?m\.?)')

# Drop column 'Batting'
batting_df = batting_df.drop(columns=['Time'])

In [6]:
batting_df["Attendace"]=batting_df["Attendace"].str.replace(",","", regex=True)
batting_df["cWPA"]=batting_df["cWPA"].str.replace("%","", regex=True)
batting_df["Attendace"] = batting_df["Attendace"].fillna(0).astype("int")
batting_df["cWPA"] = batting_df["cWPA"].fillna(0).astype("float")

#Input the mean of the venue
batting_df.loc[batting_df["Attendace"] == 0, "Attendace"] = (
    batting_df[batting_df["Attendace"] > 0]  # Filter out rows where Attendace is zero
    .groupby("Venue")["Attendace"].transform("mean")
)
batting_df.dtypes

Player Name            object
Position               object
AB                    float64
R                       int64
H                       int64
RBI                     int64
BB                      int64
SO                      int64
PA                      int64
BA                    float64
OBP                   float64
SLG                   float64
OPS                   float64
Pit                   float64
Str                   float64
WPA                   float64
aLI                   float64
WPA+                  float64
WPA-                  float64
cWPA                  float64
acLI                  float64
RE24                  float64
PO                      int64
A                       int64
Details                object
Date           datetime64[ns]
Attendace             float64
Venue                  object
Team                   object
Week-Day               object
Start Time             object
dtype: object

### Cleaning pitching table

In [7]:
pitching_df.head(5)

,Pitching,IP,H,R,ER,BB,SO,HR,ERA,BF,...,WPA,aLI,cWPA,acLI,RE24,Date,Time,Attendace,Venue,Team
0,Tyler Glasnow,5.0,2,2,2,4,3,0,3.6,20,...,0.000,1.07,0.00%,1.07,0.5,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
1,Ryan Brasier,1.0,0,0,0,0,1,0,0.0,3,...,0.048,0.63,0.03%,0.63,0.5,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
2,"Daniel Hudson, W (1-0)",1.0,1,0,0,0,1,0,0.0,4,...,0.044,0.62,0.03%,0.62,0.5,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
3,"Joe Kelly, H (1)",1.0,1,0,0,0,0,0,0.0,4,...,0.052,1.30,0.03%,1.30,0.5,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers
4,"Evan Phillips, S (1)",1.0,0,0,0,0,1,0,0.0,3,...,0.042,0.54,0.03%,0.54,0.5,"Wednesday, March 20, 2024",Start Time: 7:05 p.m. Local,"Attendance: 15,952",Venue: Gocheok Sky Dome,Los Angeles Dodgers


In [8]:
# Extract Player name of the pitching column
pitching_df['Player Name'] = pitching_df['Pitching'].str.extract(r'^([A-Za-z]+\s[A-Za-z]+)')

# Drop column 'Batting'
pitching_df = pitching_df.drop(columns=['Pitching'])

# Reorder columns: move the last two columns to the front
cols = pitching_df.columns.tolist()  # Get a list of columns
reordered_cols = cols[-1:] + cols[:-1]  # Move last two to the front
pitching_df = pitching_df[reordered_cols]

# Use loc for assignment with boolean indexing to replace mising venue
pitching_df.loc[pitching_df["Attendace"].str.startswith("Venue: ", na=False), "Venue"] = (
    pitching_df.loc[pitching_df["Attendace"].str.startswith("Venue: ", na=False), "Attendace"]
)
pitching_df.loc[pitching_df["Attendace"].str.startswith("Venue: ", na=False), "Attendace"] = np.nan

#Clean the venue name
pitching_df["Venue"]=pitching_df["Venue"].str.replace("Venue: ","",regex=True)

#Clean the attendance
pitching_df["Attendace"]=pitching_df["Attendace"].str.replace("Attendance: ","",regex=True)

#Extract the day of the week from the date
pitching_df['Week-Day'] = pitching_df['Date'].str.extract(r'^\s*(\w+)\,\s\w+\s\d+\,\s\d+$')

# Convert to datetime
pitching_df['Date'] = pd.to_datetime(pitching_df['Date'], format='%A, %B %d, %Y')

# Extract only the time
pitching_df['Start Time'] =  pitching_df['Time'].str.extract(r'(?i)\b(\d{1,2}:\d{2}\s?[ap]\.?m\.?)')

# Drop column 'Batting'
pitching_df = pitching_df.drop(columns=['Time'])

pitching_df["Attendace"]=pitching_df["Attendace"].str.replace(",","", regex=True)
pitching_df["cWPA"]=pitching_df["cWPA"].str.replace("%","", regex=True)
pitching_df["Attendace"] = pitching_df["Attendace"].fillna(0).astype("int")
pitching_df["cWPA"] = pitching_df["cWPA"].fillna(0).astype("float")

#Input the mean of the venue
pitching_df.loc[pitching_df["Attendace"] == 0, "Attendace"] = (
    pitching_df[pitching_df["Attendace"] > 0]  # Filter out rows where Attendace is zero
    .groupby("Venue")["Attendace"].transform("mean")
)
pitching_df.dtypes

Player Name            object
IP                    float64
H                       int64
R                       int64
ER                      int64
BB                      int64
SO                      int64
HR                      int64
ERA                   float64
BF                      int64
Pit                   float64
Str                   float64
Ctct                  float64
StS                   float64
StL                   float64
GB                    float64
FB                    float64
LD                    float64
Unk                   float64
GSc                   float64
IR                    float64
IS                    float64
WPA                   float64
aLI                   float64
cWPA                  float64
acLI                  float64
RE24                  float64
Date           datetime64[ns]
Attendace             float64
Venue                  object
Team                   object
Week-Day               object
Start Time             object
dtype: obj

In [9]:
pitching_df.head(5)

,Player Name,IP,H,R,ER,BB,SO,HR,ERA,BF,...,aLI,cWPA,acLI,RE24,Date,Attendace,Venue,Team,Week-Day,Start Time
0,Tyler Glasnow,5.0,2,2,2,4,3,0,3.6,20,...,1.07,0.00,1.07,0.5,2024-03-20,15952.0,Gocheok Sky Dome,Los Angeles Dodgers,Wednesday,7:05 p.m.
1,Ryan Brasier,1.0,0,0,0,0,1,0,0.0,3,...,0.63,0.03,0.63,0.5,2024-03-20,15952.0,Gocheok Sky Dome,Los Angeles Dodgers,Wednesday,7:05 p.m.
2,Daniel Hudson,1.0,1,0,0,0,1,0,0.0,4,...,0.62,0.03,0.62,0.5,2024-03-20,15952.0,Gocheok Sky Dome,Los Angeles Dodgers,Wednesday,7:05 p.m.
3,Joe Kelly,1.0,1,0,0,0,0,0,0.0,4,...,1.30,0.03,1.30,0.5,2024-03-20,15952.0,Gocheok Sky Dome,Los Angeles Dodgers,Wednesday,7:05 p.m.
4,Evan Phillips,1.0,0,0,0,0,1,0,0.0,3,...,0.54,0.03,0.54,0.5,2024-03-20,15952.0,Gocheok Sky Dome,Los Angeles Dodgers,Wednesday,7:05 p.m.


### Saving the Information in both CSV and SQL DataBase

In [10]:
#Save the information to a backup CSV

batting_df.to_csv("clean_batting.csv")
pitching_df.to_csv("clean_pitching.csv")

In [12]:
#Save the information to SQL Database

conn_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'
db = create_engine(conn_string) 
conn = db.connect()


batting_df.to_sql("Batter_data", con=conn,if_exists='replace')
pitching_df.to_sql("Pitching_data", con=conn,if_exists='replace')

conn.close()